In [1]:
import xarray as xr
from urllib.request import urlopen, Request, build_opener, HTTPCookieProcessor
import json
import ssl
import itertools
from urllib.parse import urlparse
from getpass import getpass
import base64
from urllib.error import HTTPError, URLError
import os
import math
import time
import sys
import shutil
from nsidc_functions import *

In [25]:
domain_id = 7
trial_id = 33
domains_direc = '/Users/jpflug/Documents/Projects/snowCover_comparisons/Data/ASO/selected/'
pathh = '/Users/jpflug/Documents/Projects/snowCover_comparisons/Data/MODIS/'
pathh += 'domain'+str(domain_id).zfill(2)+'_trial'+str(trial_id).zfill(2)+'/'
if not os.path.exists(pathh):
    os.makedirs(pathh)
else:
    print(f"Directory '{pathh}' already exists.")

time_start = '2022-09-01T00:00:00Z'
time_end = '2023-09-01T00:00:00Z'
buffer = 0.5

short_name = 'MOD10A1F'
version = '61'

# #### GLOBAL PARAMETERS FOR THE DOWNSCALING SCRIPTS
# polygon = ''
# filename_filter = ''
# quiet = False
# force = False
# CMR_URL = 'https://cmr.earthdata.nasa.gov'
# CMR_COLLECTIONS_URL = '{0}/search/collections.json?'.format(CMR_URL)
# CMR_URL = 'https://cmr.earthdata.nasa.gov'
# CMR_PAGE_SIZE = 2000
# CMR_FILE_URL = ('{0}/search/granules.json?'
#                 '&sort_key[]=start_date&sort_key[]=producer_granule_id'
#                 '&page_size={1}'.format(CMR_URL, CMR_PAGE_SIZE))
# FILE_DOWNLOAD_MAX_RETRIES = 3

Directory '/Users/jpflug/Documents/Projects/snowCover_comparisons/Data/MODIS/domain07_trial33/' already exists.


In [26]:
dom_ref = domains_direc+'domain'+str(domain_id).zfill(2)+'_trial'+str(trial_id).zfill(2)+'.tif'
dom_ref = xr.open_dataset(dom_ref).isel(band=0).rio.reproject(4326)

x_lims = [dom_ref.x.mean().values-buffer,dom_ref.x.mean().values+buffer]
y_lims = [dom_ref.y.mean().values-buffer,dom_ref.y.mean().values+buffer]
bounding_box = str(x_lims[0])+','+str(y_lims[0])+','+str(x_lims[1])+','+str(y_lims[1])
print(x_lims,y_lims)
print(bounding_box)

[-107.0475763902401, -106.0475763902401] [36.81518996673092, 37.81518996673092]
-107.0475763902401,36.81518996673092,-106.0475763902401,37.81518996673092


In [27]:
def cmr_download(urls, pathh, force=False, quiet=False):
    """Download files from list of urls."""
    if not urls:
        return

    url_count = len(urls)
    if not quiet:
        print('Downloading {0} files...'.format(url_count))
    credentials = None
    token = None

    for index, url in enumerate(urls, start=1):
        if not credentials and not token:
            p = urlparse(url)
            if p.scheme == 'https':
                credentials, token = get_login_credentials()

        filename = url.split('/')[-1]
        print(filename)
        if not quiet:
            print('{0}/{1}: {2}'.format(str(index).zfill(len(str(url_count))),
                                        url_count, filename))

        for download_attempt_number in range(1, FILE_DOWNLOAD_MAX_RETRIES + 1):
            if not quiet and download_attempt_number > 1:
                print('Retrying download of {0}'.format(url))
            try:
                response = get_login_response(url, credentials, token)
                length = int(response.headers['content-length'])
                try:
                    if not force and length == os.path.getsize(filename):
                        if not quiet:
                            print('  File exists, skipping')
                        # We have already downloaded the file. Break out of the
                        # retry loop.
                        break
                except OSError:
                    pass
                count = 0
                chunk_size = min(max(length, 1), 1024 * 1024)
                max_chunks = int(math.ceil(length / chunk_size))
                time_initial = time.time()
                with open(filename, 'wb') as out_file:
                    for data in cmr_read_in_chunks(response, chunk_size=chunk_size):
                        out_file.write(data)
#                         shutil.copy(filename,pathh+filename)
#                         os.remove(filename)
                        if not quiet:
                            count = count + 1
                            time_elapsed = time.time() - time_initial
                            download_speed = get_speed(time_elapsed, count * chunk_size)
                            output_progress(count, max_chunks, status=download_speed)
                if not quiet:
                    print()
                # If we get here, the download was successful and we can break
                # out of the retry loop.
                break
            except HTTPError as e:
                print('HTTP error {0}, {1}'.format(e.code, e.reason))
            except URLError as e:
                print('URL error: {0}'.format(e.reason))
            except IOError:
                raise

            # If this happens, none of our attempts to download the file
            # succeeded. Print an error message and raise an error.
            if download_attempt_number == FILE_DOWNLOAD_MAX_RETRIES:
                print('failed to download file {0}.'.format(filename))
                sys.exit(1)

In [28]:
url_list = cmr_search(short_name, version, time_start, time_end,
                      bounding_box=bounding_box, polygon=polygon,
                      filename_filter=filename_filter, quiet=quiet)

Querying for data:
	https://cmr.earthdata.nasa.gov/search/granules.json?&sort_key[]=start_date&sort_key[]=producer_granule_id&page_size=2000&short_name=MOD10A1F&version=061&version=61&temporal[]=2022-09-01T00:00:00Z,2023-09-01T00:00:00Z&bounding_box=-107.0475763902401,36.81518996673092,-106.0475763902401,37.81518996673092&provider=NSIDC_ECS

Found 366 matches.


In [29]:
cmr_download(url_list, pathh, force=force, quiet=quiet)

Earthdata username (or press Return to use a bearer token): jpflug
password: ········
MOD10A1F.A2022244.h09v05.061.2022246051414.hdf
001/732: MOD10A1F.A2022244.h09v05.061.2022246051414.hdf
  [==============================                              ]  50%  5.5MB/s    [============================================================] 100%  9.9MB/s   
MOD10A1F.A2022244.h09v05.061.2022246051414.hdf.xml
002/732: MOD10A1F.A2022244.h09v05.061.2022246051414.hdf.xml
  [============================================================] 100%  4.9MB/s   
MOD10A1F.A2022245.h09v05.061.2022247050304.hdf
003/732: MOD10A1F.A2022245.h09v05.061.2022247050304.hdf
  [==============================                              ]  50%  5.3MB/s   [============================================================] 100%  10.4MB/s   
MOD10A1F.A2022245.h09v05.061.2022247050304.hdf.xml
004/732: MOD10A1F.A2022245.h09v05.061.2022247050304.hdf.xml
  [============================================================] 100%  4.3MB

  [============================================================] 100%  4.6MB/s   
MOD10A1F.A2022264.h09v05.061.2022266055844.hdf
041/732: MOD10A1F.A2022264.h09v05.061.2022266055844.hdf
  [============================================================] 100%  3.1MB/s   
MOD10A1F.A2022264.h09v05.061.2022266055844.hdf.xml
042/732: MOD10A1F.A2022264.h09v05.061.2022266055844.hdf.xml
  [============================================================] 100%  5.8MB/s   
MOD10A1F.A2022265.h09v05.061.2022267041532.hdf
043/732: MOD10A1F.A2022265.h09v05.061.2022267041532.hdf
  [============================================================] 100%  5.1MB/s   
MOD10A1F.A2022265.h09v05.061.2022267041532.hdf.xml
044/732: MOD10A1F.A2022265.h09v05.061.2022267041532.hdf.xml
  [============================================================] 100%  3.3MB/s   
MOD10A1F.A2022266.h09v05.061.2022268035904.hdf
045/732: MOD10A1F.A2022266.h09v05.061.2022268035904.hdf
  [==================================================

  [============================================================] 100%  4.2MB/s   
MOD10A1F.A2022284.h09v05.061.2022297152329.hdf
081/732: MOD10A1F.A2022284.h09v05.061.2022297152329.hdf
  [==============================                              ]  50%  5.8MB/s   [============================================================] 100%  10.9MB/s   
MOD10A1F.A2022284.h09v05.061.2022297152329.hdf.xml
082/732: MOD10A1F.A2022284.h09v05.061.2022297152329.hdf.xml
  [============================================================] 100%  3.8MB/s   
MOD10A1F.A2022285.h09v05.061.2022297160853.hdf
083/732: MOD10A1F.A2022285.h09v05.061.2022297160853.hdf
  [==============================                              ]  50%  5.4MB/s    [============================================================] 100%  9.7MB/s   
MOD10A1F.A2022285.h09v05.061.2022297160853.hdf.xml
084/732: MOD10A1F.A2022285.h09v05.061.2022297160853.hdf.xml
  [============================================================] 100%  4.5MB/s 

  [============================================================] 100%  3.0MB/s   
MOD10A1F.A2022302.h09v05.061.2022304052012.hdf
117/732: MOD10A1F.A2022302.h09v05.061.2022304052012.hdf
  [==============================                              ]  50%  6.0MB/s   [============================================================] 100%  11.2MB/s   
MOD10A1F.A2022302.h09v05.061.2022304052012.hdf.xml
118/732: MOD10A1F.A2022302.h09v05.061.2022304052012.hdf.xml
  [============================================================] 100%  4.5MB/s   
MOD10A1F.A2022303.h09v05.061.2022305203102.hdf
119/732: MOD10A1F.A2022303.h09v05.061.2022305203102.hdf
  [==============================                              ]  50%  6.3MB/s   [============================================================] 100%  11.0MB/s   
MOD10A1F.A2022303.h09v05.061.2022305203102.hdf.xml
120/732: MOD10A1F.A2022303.h09v05.061.2022305203102.hdf.xml
  [============================================================] 100%  5.2MB/s 

  [============================================================] 100%  4.6MB/s   
MOD10A1F.A2022320.h09v05.061.2022322061247.hdf
153/732: MOD10A1F.A2022320.h09v05.061.2022322061247.hdf
  [====================                                        ]  33%  5.5MB/s    [========================================                    ]  67%  6.2MB/s    [============================================================] 100%  9.3MB/s   
MOD10A1F.A2022320.h09v05.061.2022322061247.hdf.xml
154/732: MOD10A1F.A2022320.h09v05.061.2022322061247.hdf.xml
  [============================================================] 100%  4.7MB/s   
MOD10A1F.A2022321.h09v05.061.2022323050942.hdf
155/732: MOD10A1F.A2022321.h09v05.061.2022323050942.hdf
  [==============================                              ]  50%  3.7MB/s    [============================================================] 100%  5.6MB/s   
MOD10A1F.A2022321.h09v05.061.2022323050942.hdf.xml
156/732: MOD10A1F.A2022321.h09v05.061.2022323050942.hdf.xml


...

  [==============================                              ]  50%  3.9MB/s    [============================================================] 100%  5.4MB/s   
MOD10A1F.A2023154.h09v05.061.2023156033849.hdf.xml
552/732: MOD10A1F.A2023154.h09v05.061.2023156033849.hdf.xml
  [============================================================] 100%  5.3MB/s   
MOD10A1F.A2023155.h09v05.061.2023157040355.hdf
553/732: MOD10A1F.A2023155.h09v05.061.2023157040355.hdf
  [==============================                              ]  50%  5.6MB/s    [============================================================] 100%  7.9MB/s   
MOD10A1F.A2023155.h09v05.061.2023157040355.hdf.xml
554/732: MOD10A1F.A2023155.h09v05.061.2023157040355.hdf.xml
  [============================================================] 100%  3.0MB/s   
MOD10A1F.A2023156.h09v05.061.2023160191508.hdf
555/732: MOD10A1F.A2023156.h09v05.061.2023160191508.hdf
  [==============================                              ]  50%  4.5MB/s 

  [============================================================] 100%  3.2MB/s   
MOD10A1F.A2023173.h09v05.061.2023175054547.hdf
589/732: MOD10A1F.A2023173.h09v05.061.2023175054547.hdf
  [==============================                              ]  50%  6.0MB/s   [============================================================] 100%  11.8MB/s   
MOD10A1F.A2023173.h09v05.061.2023175054547.hdf.xml
590/732: MOD10A1F.A2023173.h09v05.061.2023175054547.hdf.xml
  [============================================================] 100%  2.9MB/s   
MOD10A1F.A2023174.h09v05.061.2023176041408.hdf
591/732: MOD10A1F.A2023174.h09v05.061.2023176041408.hdf
  [============================================================] 100%  6.0MB/s   
MOD10A1F.A2023174.h09v05.061.2023176041408.hdf.xml
592/732: MOD10A1F.A2023174.h09v05.061.2023176041408.hdf.xml
  [============================================================] 100%  3.6MB/s   
MOD10A1F.A2023175.h09v05.061.2023177033958.hdf
593/732: MOD10A1F.A2023175.h09

  [============================================================] 100%  4.8MB/s   
MOD10A1F.A2023194.h09v05.061.2023200031904.hdf
631/732: MOD10A1F.A2023194.h09v05.061.2023200031904.hdf
  [==============================                              ]  50%  3.1MB/s    [============================================================] 100%  5.7MB/s   
MOD10A1F.A2023194.h09v05.061.2023200031904.hdf.xml
632/732: MOD10A1F.A2023194.h09v05.061.2023200031904.hdf.xml
  [============================================================] 100%  2.9MB/s   
MOD10A1F.A2023195.h09v05.061.2023200061758.hdf
633/732: MOD10A1F.A2023195.h09v05.061.2023200061758.hdf
  [============================================================] 100%  5.4MB/s   
MOD10A1F.A2023195.h09v05.061.2023200061758.hdf.xml
634/732: MOD10A1F.A2023195.h09v05.061.2023200061758.hdf.xml
  [============================================================] 100%  2.8MB/s   
MOD10A1F.A2023196.h09v05.061.2023201123753.hdf
635/732: MOD10A1F.A2023196.h09

  [============================================================] 100%  5.3MB/s   
MOD10A1F.A2023215.h09v05.061.2023217034009.hdf
673/732: MOD10A1F.A2023215.h09v05.061.2023217034009.hdf
  [============================================================] 100%  4.2MB/s   
MOD10A1F.A2023215.h09v05.061.2023217034009.hdf.xml
674/732: MOD10A1F.A2023215.h09v05.061.2023217034009.hdf.xml
  [============================================================] 100%  2.7MB/s   
MOD10A1F.A2023216.h09v05.061.2023220022126.hdf
675/732: MOD10A1F.A2023216.h09v05.061.2023220022126.hdf
  [============================================================] 100%  4.3MB/s   
MOD10A1F.A2023216.h09v05.061.2023220022126.hdf.xml
676/732: MOD10A1F.A2023216.h09v05.061.2023220022126.hdf.xml
  [============================================================] 100%  3.2MB/s   
MOD10A1F.A2023217.h09v05.061.2023221004843.hdf
677/732: MOD10A1F.A2023217.h09v05.061.2023221004843.hdf
  [==================================================

  [==============================                              ]  50%  4.5MB/s    [============================================================] 100%  8.1MB/s   
MOD10A1F.A2023235.h09v05.061.2023237034614.hdf.xml
714/732: MOD10A1F.A2023235.h09v05.061.2023237034614.hdf.xml
  [============================================================] 100%  4.9MB/s   
MOD10A1F.A2023236.h09v05.061.2023238034544.hdf
715/732: MOD10A1F.A2023236.h09v05.061.2023238034544.hdf
  [==============================                              ]  50%  5.1MB/s    [============================================================] 100%  7.7MB/s   
MOD10A1F.A2023236.h09v05.061.2023238034544.hdf.xml
716/732: MOD10A1F.A2023236.h09v05.061.2023238034544.hdf.xml
  [============================================================] 100%  4.4MB/s   
MOD10A1F.A2023237.h09v05.061.2023239033458.hdf
717/732: MOD10A1F.A2023237.h09v05.061.2023239033458.hdf
  [==============================                              ]  50%  4.1MB/s 